In [10]:
import pandas as pd
import numpy as np
import datetime
import pickle

In [11]:
head=["id","date_time","flag","option"]

temporal_dataset = pd.read_csv("./../data/FTDD/temporal_activity.csv",header=None)
temporal_dataset.columns=head
temporal_dataset["date_time"]=pd.to_datetime(temporal_dataset["date_time"])

In [12]:
#reduce the set
compressed_set = temporal_dataset.groupby(['id'])['flag'].apply(lambda x: "%s" % ''.join(x)).reset_index()
#i have not sorted by time but it seems to have come in order

In [13]:
t_max = temporal_dataset.groupby(['id'])['date_time'].apply(lambda x: max(x)).reset_index()
t_min =temporal_dataset.groupby(['id'])['date_time'].apply(lambda x: min(x)).reset_index()

In [14]:
compressed_set["start"]=t_min["date_time"]
compressed_set["end"]=t_max["date_time"]
compressed_set["duration"]=(compressed_set["end"]-compressed_set["start"]).astype('timedelta64[D]')

compressed_set.drop(compressed_set[compressed_set["duration"]<1].index, inplace=True)
compressed_set=compressed_set.reset_index(drop=True)

compressed_set.head()

,id,flag,start,end,duration
0,322067,NWCZ,2006-01-01 04:10:56,2007-07-04 14:35:00,549.0
1,322077,NAVCVVVYLZ,2006-01-01 07:08:00,2006-10-31 14:56:00,303.0
2,322080,NCWCCZ,2006-01-01 07:52:24,2009-08-26 22:06:00,1333.0
3,322081,NCCZ,2006-01-01 07:54:33,2007-06-06 15:25:00,521.0
4,322094,NCZ,2006-01-01 12:04:36,2006-01-02 18:59:00,1.0


In [15]:
compressed_set["label"]=(compressed_set["duration"]>60)
compressed_set["label"]=compressed_set["label"].apply(lambda x: 1 if x==False else 2)
compressed_set

,id,flag,start,end,duration,label
0,322067,NWCZ,2006-01-01 04:10:56,2007-07-04 14:35:00,549.0,2
1,322077,NAVCVVVYLZ,2006-01-01 07:08:00,2006-10-31 14:56:00,303.0,2
2,322080,NCWCCZ,2006-01-01 07:52:24,2009-08-26 22:06:00,1333.0,2
3,322081,NCCZ,2006-01-01 07:54:33,2007-06-06 15:25:00,521.0,2
4,322094,NCZ,2006-01-01 12:04:36,2006-01-02 18:59:00,1.0,1
...,...,...,...,...,...,...
60606,1116530,NCCZ,2014-12-30 10:39:00,2015-02-05 23:13:07,37.0,1
60607,1116570,NDZ,2014-12-30 12:43:00,2015-01-05 14:50:37,6.0,1
60608,1116656,NCCZ,2014-12-30 18:39:00,2015-01-02 09:20:37,2.0,1
60609,1116679,NQCQZ,2014-12-30 19:47:00,2015-01-13 14:08:07,13.0,1


In [16]:
msk = np.random.rand(len(compressed_set)) < 0.6
train_df = compressed_set[msk]
test_df = compressed_set[~msk]

In [36]:
'''
train_data_source = '../data/ag_news_csv/train.csv'
test_data_source = '../data/ag_news_csv/test.csv'

train_df = pd.read_csv(train_data_source, header=None)
test_df = pd.read_csv(test_data_source, header=None)

# concatenate column 1 and column 2 as one text
for df in [train_df, test_df]:
    df[1] = df[1] + df[2]
    df = df.drop([2], axis=1)
'''    
# convert string to lower case 
train_texts = train_df["flag"].values 
train_texts = [s.lower() for s in train_texts] 

test_texts = test_df["flag"].values 
test_texts = [s.lower() for s in test_texts] 

test_texts = list(map(lambda x: x[:7] ,test_texts))


In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [19]:


# Tokenizer
tk = Tokenizer(char_level=True)
tk.fit_on_texts(train_texts)
# If we already have a character list, then replace the tk.word_index

In [20]:
alphabet="nmearcdvyshfwlpqz"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
    
tk.word_index = char_dict.copy() 

train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

In [21]:


# Padding
train_data = pad_sequences(train_sequences, padding='post')
test_data = pad_sequences(test_texts,maxlen=train_data.shape[1], padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')



In [22]:
#=======================Get classes================
train_classes = train_df["label"].values
train_class_list = [x-1 for x in train_classes]

test_classes = test_df["label"].values
test_class_list = [x-1 for x in test_classes]

from keras.utils import to_categorical
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

## Construct Model

We implement the char_cnn_zhang model from this paper:

- Xiang Zhang, Junbo Zhao, Yann LeCun. [Character-level Convolutional Networks for Text Classification](http://arxiv.org/abs/1509.01626). NIPS 2015

The model structure:

![](https://cdn-images-1.medium.com/max/1600/0*fovAEUSdSkbsnJw5.png)

This graph may look difficult to understand. Here is the model setup. 


![](https://img-blog.csdn.net/20170721104727009)

We choose the small frame, 256 filters in convolutional layer and 1024 output units in dense layer. 

- Embedding Layer
- Six convolutional layers, and 3 convolutional layers followed by a max pooling layer
- Two fully connected layer(dense layer in keras), neuron units are 1024. 
- Output layer(dense layer), neuron units depends on classes. In this task, we set it 2. 

First we have to construct a embedding index. These characters are saved to `tk.word_index`, we could output it to see:

In [23]:
print(tk.word_index)

{'n': 1, 'm': 2, 'e': 3, 'a': 4, 'r': 5, 'c': 6, 'd': 7, 'v': 8, 'y': 9, 's': 10, 'h': 11, 'f': 12, 'w': 13, 'l': 14, 'p': 15, 'q': 16, 'z': 17}


In [24]:
vocab_size = len(tk.word_index)
vocab_size

17

We can use one-hot vector to represent these words. Because Keras use `0` for `PAD`. We add first line containing all 0 to represent the `PAD`

In [25]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

## train the model

In [26]:
test_data.shape

(24503, 157)

In [27]:
# =====================Char CNN in whole dataset=======================
# parameter
input_size = test_data.shape[1]
vocab_size = len(tk.word_index)
embedding_size = vocab_size
conv_layers = [[256, 7, 3],
               [256, 7, 3],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 2
dropout_p = 0.5
optimizer = 'adam'
loss = 'binary_crossentropy'


In [28]:

# Embedding weights
embedding_weights = []
embedding_weights.append(np.zeros(vocab_size)) 

for char, i in tk.word_index.items(): 
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
print('Load')



Load


In [29]:
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

In [30]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [35]:


# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  
# Embedding
x = embedding_layer(inputs)
# Conv
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['acc',f1_m,precision_m, recall_m])  # Adam, binary_crossentropy
model.summary()





Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 157)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 157, 17)           306       
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 151, 256)          30720     
_________________________________________________________________
activation_7 (Activation)    (None, 151, 256)          0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 50, 256)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 44, 256)           459008    
_________________________________________________________________
activation_8 (Activation)    (None, 44, 256)           0   

In [32]:
# Shuffle
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]

x_test = test_data
y_test = test_classes

In [33]:
# Training
history = model.fit(x_train, y_train,
          validation_split=0.1,
          batch_size=128,
          epochs=10,
          verbose=2)

Train on 32497 samples, validate on 3611 samples
Epoch 1/10
 - 47s - loss: 0.5635 - acc: 0.6973 - f1_m: 0.6973 - precision_m: 0.6973 - recall_m: 0.6973 - val_loss: 0.5329 - val_acc: 0.7150 - val_f1_m: 0.7157 - val_precision_m: 0.7157 - val_recall_m: 0.7157
Epoch 2/10
 - 51s - loss: 0.5335 - acc: 0.7188 - f1_m: 0.7188 - precision_m: 0.7188 - recall_m: 0.7188 - val_loss: 0.5558 - val_acc: 0.7009 - val_f1_m: 0.7020 - val_precision_m: 0.7020 - val_recall_m: 0.7020
Epoch 3/10
 - 52s - loss: 0.5249 - acc: 0.7235 - f1_m: 0.7235 - precision_m: 0.7235 - recall_m: 0.7235 - val_loss: 0.5430 - val_acc: 0.7139 - val_f1_m: 0.7167 - val_precision_m: 0.7167 - val_recall_m: 0.7167
Epoch 4/10
 - 52s - loss: 0.5204 - acc: 0.7282 - f1_m: 0.7282 - precision_m: 0.7282 - recall_m: 0.7282 - val_loss: 0.5475 - val_acc: 0.7137 - val_f1_m: 0.7144 - val_precision_m: 0.7144 - val_recall_m: 0.7144
Epoch 5/10
 - 50s - loss: 0.5139 - acc: 0.7318 - f1_m: 0.7317 - precision_m: 0.7317 - recall_m: 0.7317 - val_loss: 0.53